# Chapter 5 Case Study

We start off by a detailed look into the word2vec algorithm and examine a python implementation of the skip-gram model with negative sampling. Once the concepts underpinning word2vec are examined, we will use the Gensim package to speed up training time and investigate the translational properties of word embeddings. We will examine Glove embeddings as an alternative to word2vec. Both methods,  however, are unable to handle antonymy, polysemy and word-sense disambiguation. We will study how an embedding methods like sense2vec can better handle these issues. Lastly, we will examing document clustering by using an embeddings approach.

## Software Tools and Libraries

In this case study, we will be examining the inner operations of word2vec's skip-gram & negative sampling approach as well as GloVe embeddings with python. We will also leverage the popular nltk, gensim, glove_python, and spaCy libraries for our analysis. NLTK is a popular open-source toolkit for natural langauge processing and text analytics. The gensim library is an open-source toolkit for vector space modeling and topic modeling implemented in Python with Cython performance acceleration. The glove_python library (https://github.com/maciejkula/glove-python) is an efficient open-source implementation of GloVe in python. SpaCy is a fast open-source NLP library written in Python and Cython for part-of-speech tagging and named entity recognition.

For our analysis, we will leverage the Open American National Corpus (http://www.anc.org/), which consists of roughly 15 million spoken and written words from a variety of sources. Specifically, we will be using the subcorpus which consists of 4531 Slate magazine articles from 1996 to 2000 (approximately 4.2 million words). 


In [ ]:
# Prepping our environment

# Basics
import glob
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.feature_extraction import stop_words
import math
import re
import string

# NLTK
import nltk
from nltk import tokenize, sent_tokenize

print("Packages imported")

# 1. Exploratory Data Analysis



## 1.1. Get Data

In [ ]:
# Get slate data:  http://www.anc.org/data/oanc/contents/#slate

!cd data; tar xf slate1.tgz
!cd data; tar xf slate2.tgz
!cd data; tar xf slate3.tgz
!ls data/slate

## 1.2. Load Data

Next, we load the slate news articles into a pandas dataframe.

In [ ]:
#Load raw text files

ndf = pd.DataFrame(columns=["filename",'text'])

for datafile in glob.glob( "data/slate/*/*.txt" ):
    f=open(datafile,"r",encoding="utf-8")
    ndf = ndf.append( {'filename':datafile,'text':f.read()} ,ignore_index=True)
    f.close()

ndf.count()

To process the data with word2vec, we must first split each article into separate sentences and create a new dataframe to hold them.

In [ ]:
# Convert into dataframe of sentences

nltk.download('punkt')

df = pd.DataFrame()
for f in ndf['text']:
  sentence = sent_tokenize(f)
  df = df.append(sentence)
df.columns = ['text']
df.count()

## 1.3. Preprocessing

Word2vec needs clean word tokens for **training**, so lets clean up the sentences by preprocessing to remove punctuation, hidden characters, and uppercase characters.

In [ ]:
def process_text(text):
    
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\n", " ", text)

    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
#    stops = stop_words.ENGLISH_STOP_WORDS
#    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # Stemming
#     text = text.split()
#     stemmer = SnowballStemmer('english')
#     stemmed_words = [stemmer.stem(word) for word in text]
#     text = " ".join(stemmed_words)
    return text

# apply the above function to df['text']
df['text'] = df['text'].map(lambda x: process_text(x))

corpus = df['text'].tolist()
df.head()

Let's calculate some basic statistics on this dataset, starting with document length and sentence length.

In [ ]:
# document length histogram

pd.DataFrame([len(x.split()) for x in ndf['text']],columns= ['document length']).hist(bins=50)

In [ ]:
pd.DataFrame([len(x.split()) for x in df['text']],columns= ['sentence length']).hist(bins=50,range=(0,100))

Let's examine word-frequency by looking at the top 1000 terms in this corpus. The top 100 terms are what we typically consider stop-words, since they are common across most sentences and do not capture much, if any, semantic meaning. As we move further down, we start to see words that play a more important role in conveying the meaning within a sentence or document.

In [ ]:
words = [tokenize.word_tokenize(x) for x in corpus]
words = [y for x in words for y in x]
word_freq = pd.DataFrame(nltk.FreqDist(words).most_common(1000), columns=['word','frequency'])
print(word_freq)

In [ ]:
word_freq.plot(xlim=(0,400))

# 2.  Learning Word Embeddings

Our goal is to train a set of word embeddings for the corpus above. Let's build a skip-gram model with negative sampling, followed by a GloVe model. Before we train either model, let's see how many unique words exist in the preprocessed 4.86 million word corpus.



## 2.1. Create Dictionary of Vocabulary



In [ ]:
# Tokenize and create dictionary

class VocabWord:
    def __init__(self, word):
        self.word = word
        self.count = 0


vocab_items = []
vocab_hash = {}
word_count = 0
        
for token in ['<bol>', '<eol>']:
        vocab_hash[token] = len(vocab_items)
        vocab_items.append(VocabWord(token))

for line in df['text']:
    tokens = line.split()
    for token in tokens:
        if token not in vocab_hash:
            vocab_hash[token] = len(vocab_items)
            vocab_items.append(VocabWord(token))

        #assert vocab_items[vocab_hash[token]].word == token, 'Wrong vocab_hash index'
        vocab_items[vocab_hash[token]].count += 1
        word_count += 1

        if word_count % 1000000 == 0:
            print("\rReading word %d" % word_count)

    # Add special tokens <bol> (beginning of line) and <eol> (end of line)
    vocab_items[vocab_hash['<bol>']].count += 1
    vocab_items[vocab_hash['<eol>']].count += 1
    word_count += 2

print('Total words in training file: %d' % word_count)
print('Vocab size: %d' % len(vocab_items))

## 2.2. Word2Vec

We are now ready to train the neural network of the word2vec model. Let's define our model parameters:

*  dim = dimension of the word vectors
*  win = context window size (number of tokens)
*  start_alpha = starting learning rate
*  neg = number of samples for negative sampling
*  min_count = minimum number of mentions for a word to be included in vocabulary






In [ ]:
# Initialize parameters of neural network

dim = 100
win = 10
start_alpha = 0.05
neg = 10
min_count = 5

## 2.3. Handling Rare Words

We want to filter out rare words that have few mentions than our min_count threshold. We will be mapping all of these words to a special out-of-vocabulary token. 

In [ ]:
# truncate dictionary and map rare words to <unk> token
truncated = []
truncated.append(VocabWord('<unk>'))
unk_hash = 0

count_unk = 0
for token in vocab_items:
    if token.count < min_count:
        count_unk += 1
        truncated[unk_hash].count += token.count
    else:
        truncated.append(token)

truncated.sort(key=lambda token : token.count, reverse=True)

vocab_hash = {}
for i, token in enumerate(truncated):
    vocab_hash[token.word] = i

vocab_items = truncated
vocab_hash = vocab_hash
vocab_size = len(vocab_items)
print('Unknown vocab size:', count_unk)
print('Truncated vocab size: %d' % vocab_size)

## 2.4. Negative Sampling

To speed up training, let's create a negative sampling lookup table that we will use during training.

In [ ]:
# Create table of probabilities for negative sampling

power = 0.75
norm = sum([math.pow(t.count, power) for t in vocab_items]) # Normalizing constant

table_size = int(1e8) # Length of the unigram table
table = np.zeros(table_size, dtype=np.int)

p = 0 # Cumulative probability
i = 0
for j, unigram in enumerate(vocab_items):
    p += float(math.pow(unigram.count, power))/norm
    while i < table_size and float(i) / table_size < p:
        table[i] = j
        i += 1
        
def sample(table,count):
    indices = np.random.randint(low=0, high=len(table), size=count)
    return [table[i] for i in indices]

## 2.5. Training the Model

We are now ready to train the word2vec model. The approach is to train a two-layer (syn0, syn1) neural network by iterating over the sentences in the corpus and adjusting lawyer weights to maximize the probabilities of context words given a target word (skip-gram) with negative sampling. After completion, the weights of the hidden layer syn0 are the word embeddings that we seek.

In [ ]:
# Train skip-gram with negative sampling
import struct

# Sigmoid Function
def sigmoid(z):
    if z > 6:
        return 1.0
    elif z < -6:
        return 0.0
    else:
        return 1 / (1 + math.exp(-z))
    
# Init syn0 with uniform distribution on the interval [-0.5, 0.5]/dim
tmp = np.random.uniform(low=-0.5/dim, high=0.5/dim, size=(vocab_size, dim))
syn0 = np.ctypeslib.as_ctypes(tmp)
syn0 = np.array(syn0)


tmp = np.zeros(shape=(vocab_size, dim))
syn1 = np.ctypeslib.as_ctypes(tmp)
syn1 = np.array(syn1)
    
current_sent = 0
truncated_vocabulary = [x.word for x in vocab_items]
corpus = df['text'].tolist()

while current_sent < df.count()[0]:
    line = corpus[current_sent]
    sent = [vocab_hash[token] if token in truncated_vocabulary else vocab_hash['<unk>'] 
            for token in [['<bol>'] + line.split() + ['<eol>']]]
    for sent_pos, token in enumerate(sent):
        
        current_win = np.random.randint(low=1, high=win+1)
        context_start = max(sent_pos - current_win, 0)
        context_end = min(sent_pos + current_win + 1, len(sent))
        context = sent[context_start:sent_pos] + sent[sent_pos+1:context_end]

        for context_word in context:
            embed = np.zeros(DIM)
            classifiers = [(token, 1)] + [(target, 0) for target in table.sample(neg)]
            for target, label in classifiers:
                z = np.dot(syn0[context_word], syn1[target])
                p = sigmoid(z)
                g = alpha * (label - p)
                embed += g * syn1[target] 
                syn1[target] += g * syn0[context_word] 
            syn0[context_word] += embed

        word_count += 1
    current_sent += 1
    if current_sent % 2000 == 0:
        print("\rReading sentence %d" % current_sent)


embedding = dict(zip(truncated_vocabulary,syn0))
print("Trained embeddings")

# Save embedding

fo = open("word2vec", 'w+')
for token, vector in zip(truncated_vocabulary, syn0):
    fo.write('%s ' % token)
    for s in vector:
        fo.write(('%f '% s))
    fo.write('\n')
fo.close()

print(syn0.shape)
    

The semantic translation properties of these embeddings are noteworthy. Let's examine the cosine similarity between two similar words (man, woman) and two dissimilar words (candy, social). We would expect the similar words to exhibit higher similarity.



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print(cosine_similarity([embedding['man']],[embedding['woman']]))
print(cosine_similarity([embedding['candy']],[embedding['social']]))

## 2.6. Visualize Embedding space

We can visualize the word embeddings using the T-SNE algorithm to map the embeddings to 2D space. Note that T-SNE is a dimensionality reduction technique that preserves notions of proximity within a vector space (points close together in 2D are close in proximity in higher dimensions). The figure below shows the relationships of a 300 word sample from the vocabulary.  

In [ ]:
from sklearn.manifold import TSNE
from matplotlib import pylab

words = truncated_vocabulary[200:400]
embeddings = syn0
words_embedded = TSNE(n_components=2).fit_transform(embeddings[200:400])

pylab.figure(figsize=(10, 10))
for i, label in enumerate(words):
  x, y = words_embedded[i, :]
  pylab.scatter(x, y)
  pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                 ha='right', va='bottom')
pylab.show()

## 2.7. Using the Gensim package

The python code above is useful for understanding principles, but is not the fastest to run. The original word2vec package was written in C++ to faciliate rapid training speed over multiple cores. The gensim package provides an API to the word2vec library, as well as several useful methods to examine vectors neighborhoods. Let's see how we can use gensim to train on the sample data corpus.

In [ ]:
!pip install -q gensim==3.2.0

import gensim


Gensim expects us to provide a set of documents as a list of list of tokens. We will call the simple_preprocess() method of gensim to remove punctuation, special and uppercase characters.

In [ ]:
documents = [gensim.utils.simple_preprocess(df['text'].iloc[i]) for i in range(len(df))]

With the wrapper api provided by the gensim package, training word2vec is as simple as defining a model and passing the set of training documents.

In [ ]:
model = gensim.models.Word2Vec(documents,
                                size=100,
                                window=10,
                                min_count=2,
                                workers=10)
model.train(documents, total_examples=len(documents), epochs=10)

## 2.8. Similarity

Let's assess the quality of the learned word embeddings by examining word neighborhoods. If we look at the most similar words to "man" or "book", we find highly similar words in their neighborhoods. So far so good.

In [ ]:
model.wv.most_similar("man",topn=5)

In [ ]:
model.wv.most_similar("book",topn=5)

Let's look at some polysemic words. The similar words to the word "bass" reflect the music definition of bass. That is, they only capture a single word sense (there are no words related to the aquatic definition of bass). Similarly, words similar to "bank" all reflect its financial word sense, but no seashores or riverbeds. This is one of the major shortcomings of word2vec. 

In [ ]:
model.wv.most_similar("bass",topn=5)

In [ ]:
model.wv.most_similar("bank",topn=5)

We can examine the semantic translation properties in more detail with some vector algebra. If we start with the word 'son' and subtract 'man' and add 'woman', we indeed find that 'daughter' is the closest word to the resulting sum. Similarly, if we invert the operation and start with the word 'daughter' and subtract 'woman' and add 'man', we find that 'son' is closest to the sum. Note that reciprocity is not guaranteed with word2vec.

In [ ]:
model.wv.similar_by_vector(model.wv['son']-model.wv['man']+model.wv['woman'],topn=5)

In [ ]:
model.wv.similar_by_vector(model.wv['daughter']-model.wv['woman']+model.wv['man'],topn=5)

We can also see that word2vec captures geographic similarities by taking the word 'paris', subtracting 'france' and adding 'russia'. There resulting sum is close to what we expect - 'moscow'.

In [ ]:
model.wv.similar_by_vector(model.wv['paris']-model.wv['france']+model.wv['russia'],topn=5)

We have previously discussed that word embeddings generated by word2vec are unable to distinguish antonyms, as these words often share the same context words in normal usage and consequentially have learned embeddings close to each other. For instance, the most similar word to 'large' is 'small', or the most similar word to 'hard' is 'easy'. Antonymy is hard! 

In [ ]:
model.wv.most_similar("large",topn=5)

In [ ]:
model.wv.most_similar("hard",topn=5)

## 2.9. GloVe

Whereas word2vec captures the local context of words within sentences, GloVe embeddings can additionally account for global context across the corpus. Let's take a deeper dive on how to calculate GloVe embeddings. We begin by building a vocabulary dictionary from the corpus.

In [ ]:
# Build vocab

from collections import Counter

vocab_count = Counter()

for line in corpus:
    tokens = line.strip().split()
    vocab_count.update(tokens)

vocab = {word: (i, freq) for i, (word, freq) in enumerate(vocab_count.items())}

## 2.10. Co-occurence matrix

Let's build the word co-occurence matrix from the corpus.  Note that word occurrences go both ways, from the main word to context, and vice versa. For smaller values of the context window, this matrix is expected to be sparse.

In [ ]:
# Build co-occurrence matrix

from scipy import sparse

min_count = 10
window_size = 5

vocab_size = len(vocab)
id2word = dict((i, word) for word, (i, _) in vocab.items())
occurrence = sparse.lil_matrix((vocab_size, vocab_size),dtype=np.float64)

for i, line in enumerate(corpus):
    tokens = line.split()
    token_ids = [vocab[word][0] for word in tokens]

    for center_i, center_id in enumerate(token_ids):
        # Collect all word IDs in left window of center word
        context_ids = token_ids[max(0, center_i - window_size) : center_i]
        contexts_len = len(context_ids)

        for left_i, left_id in enumerate(context_ids):
            # Distance from center word
            distance = contexts_len - left_i

            # Weight by inverse of distance between words
            increment = 1.0 / float(distance)

            # Build co-occurrence matrix symmetrically (pretend we
            # are calculating right contexts as well)
            occurrence[center_id, left_id] += increment
            occurrence[left_id, center_id] += increment
    if i % 10000 == 0:
        print("Processing sentence %d" % i)

# Create a generator for efficient calculation

def occur_matrix(vocab, coccurrence, min_count):
  for i, (row, data) in enumerate(zip(coccurrence.rows, coccurrence.data)):
    if min_count is not None and vocab[id2word[i]][1] < min_count:
        continue

    for data_idx, j in enumerate(row):
        if min_count is not None and vocab[id2word[j]][1] < min_count:
            continue

        yield i, j, data[data_idx]

print("Creating co-occurrence matrix generator")
comatrix = occur_matrix(vocab, occurrence, min_count)


## 2.11. GloVe training

We can now train the embeddings by iterating over the documents (sentences) in the corpus.





In [ ]:
# Train Glove Embeddings

from random import shuffle
from math import log
import pickle

iterations = 30
dim = 100
learning_rate = 0.05
x_max = 100
alpha = 0.75

vocab_size = len(vocab)
W = (np.random.rand(vocab_size * 2, dim) - 0.5) / float(dim + 1)
biases = (np.random.rand(vocab_size * 2) - 0.5) / float(dim + 1)

gradient_squared = np.ones((vocab_size * 2, dim), dtype=np.float64)
gradient_squared_biases = np.ones(vocab_size * 2, dtype=np.float64)

data = [(W[i_main], W[i_context + vocab_size],
             biases[i_main : i_main + 1],
             biases[i_context + vocab_size : i_context + vocab_size + 1],
             gradient_squared[i_main], gradient_squared[i_context + vocab_size],
             gradient_squared_biases[i_main : i_main + 1],
             gradient_squared_biases[i_context + vocab_size
                                     : i_context + vocab_size + 1],
             cooccurrence)
            for i_main, i_context, cooccurrence in comatrix]

for i in range(iterations):
    global_cost = 0
    shuffle(data)
    for (v_main, v_context, b_main, b_context, gradsq_W_main, gradsq_W_context,
         gradsq_b_main, gradsq_b_context, cooccurrence) in data:

        weight = (cooccurrence / x_max) ** alpha if cooccurrence < x_max else 1

        cost_inner = (v_main.dot(v_context)
                      + b_main[0] + b_context[0]
                      - log(cooccurrence))
        cost = weight * (cost_inner ** 2)
        global_cost += 0.5 * cost

        grad_main = weight * cost_inner * v_context
        grad_context = weight * cost_inner * v_main
        grad_bias_main = weight * cost_inner
        grad_bias_context = weight * cost_inner

        v_main -= (learning_rate * grad_main / np.sqrt(gradsq_W_main))
        v_context -= (learning_rate * grad_context / np.sqrt(gradsq_W_context))

        b_main -= (learning_rate * grad_bias_main / np.sqrt(gradsq_b_main))
        b_context -= (learning_rate * grad_bias_context / np.sqrt(
                gradsq_b_context))

        gradsq_W_main += np.square(grad_main)
        gradsq_W_context += np.square(grad_context)
        gradsq_b_main += grad_bias_main ** 2
        gradsq_b_context += grad_bias_context ** 2
    print('iteration = ',i,' cost = ',global_cost)
    
# Save model

with open("glove","wb") as f:
    pickle.dump(W, f, protocol=2)

The learned weight matrix consists of two sets of vectors, one if the word is in the main word position and one for the context word position. We will average them to generate the final GloVe embeddings for each word.

In [ ]:
# Merge main and context word vectors by taking average

def merge_vectors(W, merge_fun=lambda m, c: np.mean([m, c], axis=0)):

    vocab_size = int(len(W) / 2)
    for i, row in enumerate(W[:vocab_size]):
        merged = merge_fun(row, W[i + vocab_size])
        merged /= np.linalg.norm(merged)
        W[i, :] = merged

    return W[:vocab_size]

embedding = merge_vectors(W)
len(embedding)

##2.12. GloVe Vector Similarity

Let's examine the translational properties of these vectors. We define a simple function that returns the 5 most similar words to the word "man".

In [ ]:
# Find most similar words

def most_similar(W, vocab, id2word, word, n=5):
    """
    Find the `n` words most similar to the given `word`. The provided
    `W` must have unit vector rows, and must have merged main- and
    context-word vectors (i.e., `len(W) == len(word2id)`).
    Returns a list of word strings.
    """

    assert len(W) == len(vocab)

    word_id = vocab[word][0]

    dists = np.dot(W, W[word_id])
    top_ids = np.argsort(dists)[::-1][:n + 1]

    return [(id2word[id],dists[id]) for id in top_ids if id != word_id][:n]

for term in most_similar(embedding, vocab, id2word, 'man', 5): print(term)

Interestingly, the similarirty results fall into two categories. Whereas "woman" and "girl" have similar semantic meaning to 'man', the words "dead" and "young" do not. But these words do co-occur often, with phrases such as "young man" or "dead man". GloVe embeddings can capture both contexts. We can see this when we visualize the embeddings using T-SNE.

In [ ]:
from sklearn.manifold import TSNE
from matplotlib import pylab

words = list(vocab.keys())[200:400]
words_embedded = TSNE(n_components=2).fit_transform(embedding[200:400])

pylab.figure(figsize=(10, 10))
for i, label in enumerate(words):
  x, y = words_embedded[i, :]
  pylab.scatter(x, y)
  pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                 ha='right', va='bottom')
pylab.show()

## 2.13. Using the Glove Package

While useful, our python implemention is too slow to run with a large corpus. The 'glove' and 'glove_python' libraries are python packages that implement the GloVe algoritm efficiently. Let's retrain our embeddings using the 'glove_python' package.

In [ ]:
!pip install glove_python

from glove import Corpus, Glove

To do so, we begin by defining a corpus of documents. 

In [ ]:
corpus = Corpus()

corpus.fit(documents, window=5)

We then learn embeddings from this corpus.

In [ ]:
glove = Glove(no_components=100, learning_rate=0.05)

glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Let's asses the quality of these embeddings by examining a few words.

In [ ]:
glove.most_similar('man',number=6)

In [ ]:
glove.most_similar('nice',number=6)

In [ ]:
glove.most_similar('apple',number=6)

Once again, the most similar words exhibit both semantic similarity as well as high co-occurrence probability. Even with the additional context, GloVe embeddings still lack the ability to handle antonyms and word sense disambiguation. 

# 3. Document Clustering

The use of word embeddings provides a useful and efficient means for document clustering in comparison to traditional approaches such as LSA or LDA. The simplest approach is a bag-of-words method where a document vector is created by averaging the vectors of each of the words in the document. Let's take our Slate corpus and see what we can find with this approach.



##3.1. Document vectors

We create a set of document vectors by adding the vectors of each word in the document and dividing by the total number of words.

In [ ]:
documents = [gensim.utils.simple_preprocess(ndf['text'].iloc[i]) for i in range(len(ndf))]
corpus = Corpus()
corpus.fit(documents, window=5)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=10, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
print("Glove embeddings trained.")

doc_vectors = []
for doc in documents:
    vec = np.zeros(glove.word_vectors[glove.dictionary['the']].shape)
    for token in doc:
      vec += glove.word_vectors[glove.dictionary[token]]
    if len(doc) > 0:
      vec = vec/len(doc)
    doc_vectors.append(vec)
    
print("Processed documents = ",len(doc_vectors))

## 3.2. Cluster Analysis

Let's visualize these embeddings and see if we can spot any clusters.

In [ ]:
from sklearn.manifold import TSNE
from matplotlib import pylab

docs_embedded = TSNE(n_components=2).fit_transform(doc_vectors)

pylab.figure(figsize=(10, 10))
for i, label in enumerate(docs_embedded):
  x, y = docs_embedded[i, :]
  pylab.scatter(x, y)
pylab.show()

# 4. Word Sense Disambiguation

Word sense disambiguation is an important task in computational linguistics. However, word2vec or GloVe embeddings map words to a single embedding vector, and therefore lack the ability to disambiguate between multiple senses of words. The sense2vec algorithm is an improved approach that can deal with polysemy or antonymy through supervised disambiguation. Moreover, sense2vec is computationally inexpensive and can be implemented as a preprocessing task prior to training a word2vec or GloVe model. To see this, let's apply the sense2vec algoritm to our corpus by leveraging the spaCy library to generate part-of-speech labels that will serve as our supervised disambiguation labels.


## 4.1. Load spaCy

In [ ]:
!pip install spacy
!python -m spacy download en

import spacy
nlp = spacy.load('en',disable=['parser', 'ner'])

print('Model loaded')

## 4.2. Supervised Disambiguation Annotations

Let's process the sentences in our corpus using the spaCy NLP annotations.

In [ ]:
corpus = df['text'].tolist()
print("Number of docs = ",len(corpus))

docs = []
count = 0
for item in corpus:
  docs.append(nlp(item))
  count += 1
  if count % 10000 == 0:
    print("Processed document #",count)

## 4.3. POS Label Annotation

We will create a separate corpus where each word is augmented by its part-of-speech label. For instance, the word 'he' is mapped to 'he_PRON'.

In [ ]:
sense_corpus = [[x.text+"_"+x.pos_ for x in y] for y in docs]
print(corpus[0])
print(' '.join(sense_corpus[0]))

## 4.4. Train using word2vec

With the new preprocessed corpus, we can proceed with training word2vec.

In [ ]:
model = gensim.models.Word2Vec(sense_corpus,
                                size=100,
                                window=10,
                                min_count=2,
                                workers=10)
model.train(sense_corpus, total_examples=len(sense_corpus), epochs=10)

## 4.5. Evaluate Word Sense Disambiguation

We can use this trained model to look at how words like 'run' or 'lie' can be disambiguated based on their part-of-speech.

In [ ]:
model.wv.most_similar("run_NOUN",topn=5)

In [ ]:
model.wv.most_similar("run_VERB",topn=5)

In [ ]:
model.wv.most_similar("lie_NOUN",topn=5)

In [ ]:
model.wv.most_similar("lie_VERB",topn=5)